In [5]:
import requests
import base64
from starlette.applications import Starlette
from starlette.responses import HTMLResponse, JSONResponse
from starlette.middleware.cors import CORSMiddleware
import uvicorn, aiohttp, asyncio

# application init
app = Starlette()
app.add_middleware(CORSMiddleware, allow_origins=['*'], allow_headers=['X-Requested-With', 'Content-Type'])

subscription_key = "1de9548a300042d1a8c5bf4d1f8233b6"
vision_base_url = "https://westcentralus.api.cognitive.microsoft.com/vision/v2.0/"
text_recognition_url = vision_base_url + "read/core/asyncBatchAnalyze"
headers = {'Ocp-Apim-Subscription-Key': subscription_key, 'Content-Type': 'application/octet-stream'}

@app.route('/')
def index(request):
    return HTMLResponse("Hello World")

@app.route('/getText', methods=['POST'])
async def get_text(request):
    
#     req_form = await request.form()
#     data = await (req_form['file'].read())
    
    req_json = await request.json()
    data = base64.b64decode(req_json["image"])
    
    response = requests.post(
        text_recognition_url,
        headers=headers,
        data=data)
    
    response.raise_for_status()
    analysis = {}
    poll = True
    
    while (poll):
        response_final = requests.get(response.headers["Operation-Location"], headers=headers)
        analysis = response_final.json()

        if ("recognitionResults" in analysis):
            poll = False
        if ("status" in analysis and analysis['status'] == 'Failed'):
            poll = False
    
    if ("recognitionResults" in analysis):
        texts = [line["text"] for line in analysis["recognitionResults"][0]["lines"]]
    
    return HTMLResponse(str(texts))


if __name__ == '__main__':
    uvicorn.run(app=app, host='0.0.0.0', port=1997)

INFO: Started server process [13720]
INFO: Waiting for application startup.
INFO: Uvicorn running on http://0.0.0.0:1997 (Press CTRL+C to quit)
INFO: ('127.0.0.1', 50152) - "POST /getText HTTP/1.1" 200
INFO: Shutting down
INFO: Waiting for application shutdown.
INFO: Finished server process [13720]
